In [2]:
# Basic
import numpy as np
from matplotlib import pyplot as plt
from scipy.linalg import pinv2
from tqdm import tqdm
import pandas as pd
import itertools

# For Dataset
from collections import deque, namedtuple
import random

# For solvers
from qpsolvers import solve_qp
from scipy.integrate import solve_ivp
from casadi import *
import casadi

# For estimators
from torch import nn
import torch
import torch.nn.functional as F

In [3]:
from utils.system import AAC
#from utils.controller2 import *
from utils.dataset import *
#from utils.estimator2 import *
from utils.normalizer import *
from utils.functions import *


In [25]:
from qpsolvers import solve_qp
from scipy.integrate import solve_ivp
from casadi import *

from qpsolvers import solve_qp
from scipy.integrate import solve_ivp
from casadi import *

class LCBF:
    def __init__(self, m_nom, ca_nom, cd_nom, f0_nom, f1_nom, f2_nom, v_lead_nom, v_des, Th, clf_rate, cbf_rate, p_slack):
        self.g = 9.81
        self.m = m_nom
        self.ca = ca_nom
        self.cd = cd_nom
        self.f0 = f0_nom
        self.f1 = f1_nom
        self.f2 = f2_nom
        self.v_lead = v_lead_nom 
        self.v_des = v_des
        self.Th = Th
        
        self.p_slack = p_slack
        self.cbf_rate = cbf_rate
        self.clf_rate = clf_rate
        
        self.k1 = 0
        
    def clf(self, x):
        v = x[1]        
        V = (v - self.v_des)**2
        return V

    def dclf(self, x, u):
        v = x[1]
        Fr = self.f0 * v**2 + self.f1 * v + self.f2
        dV = (v - self.v_des)*(2/self.m*(u - Fr))
        return dV
        
    def cbf(self, x, isMaxCD=False):
        v = x[1]
        z = x[2]
        if isMaxCD:
            h = z - self.Th * v - 0.5  * (self.v_lead - v)**2 / (self.cd * self.g)
        else:
            h = z - self.Th * v 
        return h
        
    def dcbf(self, x, u, isMaxCD=False):
        v = x[1]
        z = x[2]
        
        Fr = self.f0 * v**2 + self.f1 * v + self.f2
        if isMaxCD:
            dh = 1/self.m * (self.Th + (v - self.v_lead)/self.cd/self.g ) * (Fr - u) + (self.v_lead - v)
        else:
            dh = self.Th/self.m * (Fr - u) + self.v_lead - v
        return dh


        
    def compute_controller(self, x, u_ref, estimator, weight, t = None, normalizer = None):
        # Symbolic values
        u = SX.sym('u')
        slack = SX.sym('slack')

        # CBF-CLF calculation                
        V = self.clf(x) # Numeric
        dV = self.dclf(x, u) # Symbolic

        h = self.cbf(x) # Numeric
        dh = self.dcbf(x, u) # Symbolic
       
        if normalizer:
            x_n = normalizer['x'].update(x)
            u_n = normalizer['u'].normalize(u)
            
            
            dhe_n = estimator.forward(x_n, u_n, t)
            if dhe_n is None:
                dhe = 0
            else:
                dhe = normalizer['dhe'].denormalize(dhe_n)
            
            #dhe = taylor(dhe, u, self.k1, 2)  
        else:
            dhe_n = estimator.forward(x, u, t)
            if dhe_n is None:
                dhe = 0
            else:
                dhe = dhe_n
    
        
        # Estimator
        dS = dh + dhe
        
        # QP optimizer
        weight_input = 2/self.m**2
        fqp = (u_ref - u)**2 * weight_input + self.p_slack * slack**2
        gqp = vertcat( -dV - self.clf_rate*V + slack, dS + self.cbf_rate * h)     
        qp = {'x': vertcat(u,slack), 'f':fqp, 'g':gqp}
        S = nlpsol('S', 'ipopt', qp,{'verbose':False,'print_time':False, "ipopt": {"print_level": 0}})
        r = S(lbg=0, lbx = -self.m*self.cd*self.g/5000, ubx = self.m*self.ca*self.g/5000)
        
        # Solutions
        if normalizer:
            k = r['x'].elements()[0]
            _ = normalizer['u'].update(k)
        else:
            k = r['x'].elements()[0]
            
        self.k1 = k
        
        
        
        slack_sol = r['x'].elements()[1]

        # Create a Function to evaluate expression
        dh_f = Function('f',[u],[dh])
        dhe_f = Function('f',[u],[dhe])
        dV_f = Function('f',[u],[dV])
        dS_f = Function('f',[u],[dS])

        # Evaluate numerically
        dh = dh_f(k).elements()[0]
        dhe = dhe_f(k).elements()[0]
        dS = dS_f(k).elements()[0]
        dV = dV_f(k).elements()[0]
        
        

        return k, slack_sol, V, dV, h, dh, dhe, dS



class PID:
    def __init__(self, x_dim, u_dim, Kp, Kd, Ki, dT):
        self.x_dim = x_dim
        self.u_dim = u_dim 

        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        
        self.dT = dT
        
        self.e = np.zeros([x_dim,1])
        self.de = np.zeros([x_dim,1])
        self.ei = np.zeros([x_dim,1])
        
        # e(t-1) 
        self.e_1 = np.zeros([x_dim,1])
        
    def update(self, e):
        self.e = e
        
        # Compute derivative
        self.de = (self.e - self.e_1)/self.dT
        
        # Compute integral
        self.ei =  self.ei + self.dT*(self.e - self.e_1)/2
        
        u = self.Kp.dot(self.e) + self.Kd.dot(self.de)  + self.Ki.dot(self.ei)
        
        self.e_1 = self.e
        
        return u
        

    def reset(self):
        self.e = np.zeros([self.x_dim,1])
        self.de = np.zeros([self.x_dim,1])
        self.ei = np.zeros([self.x_dim,1])
        
        # e(t-1) 
        self.e_1 = np.zeros([self.x_dim,1])
        
        

In [29]:
class ELM: 
    def __init__(self, input_size, hidden_size = 100, output_size = 1):
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        
        self.H = nn.Sequential(nn.Linear(input_size, self.hidden_size),                   nn.Sigmoid(),
                               )
        
        self.H.requires_grad_(False)
        
        self.elm = nn.Linear(self.hidden_size, self.output_size)
        
        self.H.apply(self.weights_init)
        self.elm.apply(self.weights_init)
        
        self.model = nn.Sequential(self.H, self.elm)
        

    def weights_init(self, m):
        classname = m.__class__.__name__
        if classname.find('Linear') != -1:
            m.weight.data.normal_(0.0, 1.0)
            m.bias.data.fill_(0.0)


    def forward(self, x, train = False):
        x = torch.from_numpy(np.array(x)).float()
        
        z = self.model(x)
        
        if not train:
            z = z.detach().float().item()
            
        return z
    
    
class ELMs:
    def __init__(self, input_size, hidden_size, output_size, time_th, dt, lrate_pre = 1e-4, lrate_post = 5e-3 ):
        
        self.time_th = time_th
        self.dt = dt
        
        self.lr_pre = lrate_pre
        self.lr_post = lrate_post
        
        self.first_trained = False
        
        self.hidden_size = hidden_size
        self.e_f = ELM(input_size, hidden_size, output_size)
        self.e_g = ELM(input_size, hidden_size, output_size)
        
        self.opts_pre = {'e_f': torch.optim.Adam(self.e_f.model.parameters(), lr = self.lr_pre), 
                         'e_g': torch.optim.Adam(self.e_g.model.parameters(), lr = self.lr_pre)}
        
        self.opts_post = {'e_f': torch.optim.Adam(self.e_f.model.parameters(), lr = self.lr_post), 
                          'e_g': torch.optim.Adam(self.e_g.model.parameters(), lr = self.lr_post)}
        

    def forward(self, x, u, t, train=False):
        if self.first_trained: # t / self.dt > self.time_th and 
            ef = self.e_f.forward(x, train)
            eg = self.e_g.forward(x, train)
            return ef + eg * u          
        else:
            return None
        
    def train(self, t, data):
        if t / self.dt >= self.time_th:
            if not self.first_trained:
                opts = self.opts_pre
                self.first_trained = True
                epochs = 50
            else:
                opts = self.opts_post
                epochs = 1
                
            sample = data.get_D(t)
            
            for epoch in range(epochs):
                #running_loss = 0
                for x_i, k_i, dhe_real_i in zip(sample.x, sample.k, sample.dhe_real):
                    S_i = self.forward(x_i, k_i, t, train=True)
                    loss = F.mse_loss(torch.tensor(dhe_real_i),S_i)
                    self.e_f.model.zero_grad()
                    self.e_g.model.zero_grad()
                    
                    #running_loss += loss.item()

                    loss.backward()
                    opts['e_f'].step()
                    opts['e_g'].step()

In [24]:
from collections import deque, namedtuple
import random

class ELMDataset:
    def __init__(self, dt, features = ('x'), time_th = 0.5, maxlen = 5):
        self.time_th = time_th
        self._maxlen = maxlen
        self.D_pre = deque()
        self.D_post = deque(maxlen = self._maxlen)
        self.dt = dt
        
        self.trans = namedtuple('trans',
                                    features)
        
    def reset(self):
        self.D_pre = deque()
        self.D_post = deque(maxlen = self._maxlen)
    
    def update(self, t, *args):
        if t/self.dt < self.time_th:
            self.D_pre.append(self.trans(*args))
            self.D_post.append(self.trans(*args))       
        else:
            self.D_post.append(self.trans(*args))
        
    def shuffle(self):
        random.shuffle(self.D_post)
        
    def get_D(self, t):
        if t/self.dt < self.time_th:
            return self.trans(*zip(*self.D_pre))
        else:
            return self.trans(*zip(*self.D_post))
        


In [33]:
dt = 0.01
simTime = 20

# Real parameters
v_lead = 20
v_des = 24
m  = 1650.0
g = 9.81

# 
f0 = 0.1
f1 = 5
f2 = 0.25


c_a = 0.3
c_d = 0.3
Th = 1.8

# Nominal parameters
f0_nom = 10*f0
f1_nom = 10*f1
f2_nom = 10*f2

p_slack = 2e-2
clf_rate = 5
cbf_rate = 5.

# Initial state
p0 = 0
v0 = 20
z0 = 40

x = [p0, v0 ,z0]




In [31]:
def c2l(d):
    if type(d) == float or type(d) == int or type(d) == np.float64:
        return [d]
    if type(d) == list:
        return d
    if type(d) == np.ndarray:
        return d.tolist()
    

In [37]:
# System
aac = AAC(m, c_d, f0, f1, f2, v_lead)
derivator = Derivator(dt)

# Controller
cont = LCBF(0.8*m, c_a, c_d, f0_nom, f1_nom, f2_nom, v_lead, v_des, Th, clf_rate, cbf_rate, p_slack)

# Estimator
input_size = 3 
hidden_size = 100
output_size = 1


learned_ratio = 1.5
time_th = learned_ratio* hidden_size


weights = 0.2

# Normalizers



# PID control reference
x_dim = 3
u_dim = 1

kp = np.array([[0, 1.0e3, 0]])/5000
kd = np.array([[0, 0.1, 0]])/5000
ki = np.array([[0, 1.0e3, 0]])/5000

pid = PID(x_dim, u_dim, kp, kd, ki, dt)



####
# Filename
# OSELM: lr_pre + lr_post + z0 + v0 + func
lr_pres =  [1e-3]#[1e-2, 1e-3]
lr_posts =  [1e-3]#[1e-2]
z0s = [30]#[30, 34, 38]
v0s = [20]
funcs = [square]

data_dir = 'data/oselm'

cases = len(list(itertools.product(lr_pres, lr_posts, z0s, v0s, funcs)))
pbar = tqdm(total=cases*simTime/dt)

for lr_pre, lr_post, z0, v0, func in itertools.product(lr_pres, lr_posts, z0s, v0s, funcs):
    x = [0, v0, z0]
    fn = "{}_{}_{}_{}_{}.csv".format(lr_pre, lr_post, z0, v0, func.__name__)
    
    # Database
    column_names = ['p', 'v', 'z', 'u','u_ref','V','h','dhe_real','dhe','slack']
    df = pd.DataFrame(columns=column_names,dtype=object)

    # save file .csv
    path = os.path.join(data_dir, fn)
    df.to_csv(path, index=False)
    
    # Estimator
    estimator = ELMs(input_size, hidden_size, output_size, time_th, dt, lr_pre, lr_post)
    
    ## Dataset
    d = ELMDataset(dt, ('x', 'k', 'dhe_real'), time_th )

    for t in np.arange(0, simTime, dt): #simTime
        pbar.update(1)
        # Get reference control
        e = np.array([[0], [v_des], [0]]) - np.expand_dims(x, axis = 1)
        u_ref = pid.update(e)
        u_ref = u_ref[0,0]
       
        unct = func(t)
        aac.v_lead = v_lead + unct  # lead_vehicle

        # Control Input
        k, slack_sol, V, dV, h, dh, dhe, dS = cont.compute_controller(x, u_ref, estimator, weights, t) 
       
        # One step propagation in the system
        x_n = aac.update(x, k, t, dt)

        dh_real = derivator.update(h)
        dhe_real = dh_real - dh

        
        row = c2l(x) + c2l(k) + c2l(u_ref) + c2l(V) + c2l(h) + c2l(dhe_real) + c2l(dhe) + c2l(slack_sol)
        df_row = pd.DataFrame(dict(zip(column_names, row)), index = [0])
        
        df.append(df_row, sort = False).to_csv(path, index=False, mode = 'a', header=False)

        x = x_n
        
        d.update(t, x, k, dhe_real)

        estimator.train(t, d)
                    
pbar.close()
    
        
        

 



  0%|          | 0/2000.0 [00:00<?, ?it/s]

  0%|          | 1/2000.0 [00:00<15:14,  2.19it/s]

  0%|          | 7/2000.0 [00:00<02:07, 15.63it/s]

  1%|          | 16/2000.0 [00:00<00:58, 33.88it/s]

  1%|          | 23/2000.0 [00:00<00:47, 42.02it/s]

  1%|▏         | 29/2000.0 [00:00<00:42, 46.73it/s]

  2%|▏         | 36/2000.0 [00:00<00:38, 51.52it/s]

  2%|▏         | 43/2000.0 [00:01<00:36, 53.61it/s]

  2%|▏         | 49/2000.0 [00:01<00:36, 52.83it/s]

  3%|▎         | 55/2000.0 [00:01<00:35, 54.55it/s]

  3%|▎         | 64/2000.0 [00:01<00:31, 62.19it/s]

  4%|▎         | 73/2000.0 [00:01<00:28, 68.36it/s]

  4%|▍         | 81/2000.0 [00:01<00:29, 64.39it/s]

  4%|▍         | 88/2000.0 [00:01<00:31, 61.00it/s]

  5%|▍         | 95/2000.0 [00:01<00:32, 59.36it/s]

  5%|▌         | 102/2000.0 [00:02<00:31, 60.12it/s]

  5%|▌         | 109/2000.0 [00:02<00:30, 62.36it/s]

  6%|▌         | 116/2000.0 [00:02<00:30, 61.97it/s]

  6%|▋         | 125/2000.0 [00:02<00:27, 68.44it/s]

 43%|████▎     | 860/2000.0 [00:15<00:23, 47.98it/s]

 43%|████▎     | 865/2000.0 [00:15<00:25, 44.72it/s]

 44%|████▎     | 871/2000.0 [00:15<00:23, 47.51it/s]

 44%|████▍     | 878/2000.0 [00:15<00:21, 51.47it/s]

 44%|████▍     | 884/2000.0 [00:16<00:20, 53.24it/s]

 45%|████▍     | 891/2000.0 [00:16<00:19, 56.32it/s]

 45%|████▍     | 898/2000.0 [00:16<00:18, 58.83it/s]

 45%|████▌     | 904/2000.0 [00:16<00:20, 54.39it/s]

 46%|████▌     | 910/2000.0 [00:16<00:20, 54.14it/s]

 46%|████▌     | 916/2000.0 [00:16<00:22, 49.07it/s]

 46%|████▌     | 922/2000.0 [00:16<00:22, 48.76it/s]

 46%|████▋     | 927/2000.0 [00:16<00:22, 47.23it/s]

 47%|████▋     | 934/2000.0 [00:17<00:20, 52.16it/s]

 47%|████▋     | 941/2000.0 [00:17<00:18, 55.77it/s]

 47%|████▋     | 948/2000.0 [00:17<00:18, 57.78it/s]

 48%|████▊     | 954/2000.0 [00:17<00:19, 53.31it/s]

 48%|████▊     | 960/2000.0 [00:17<00:20, 50.71it/s]

 48%|████▊     | 967/2000.0 [00:17<00:18, 54.46it/s]

 49%|████▊     | 974/2000.0 

 86%|████████▋ | 1727/2000.0 [00:30<00:05, 49.53it/s]

 87%|████████▋ | 1733/2000.0 [00:31<00:05, 51.71it/s]

 87%|████████▋ | 1740/2000.0 [00:31<00:04, 55.02it/s]

 87%|████████▋ | 1747/2000.0 [00:31<00:04, 56.81it/s]

 88%|████████▊ | 1754/2000.0 [00:31<00:04, 58.76it/s]

 88%|████████▊ | 1761/2000.0 [00:31<00:03, 60.18it/s]

 88%|████████▊ | 1768/2000.0 [00:31<00:03, 59.44it/s]

 89%|████████▊ | 1774/2000.0 [00:31<00:03, 58.54it/s]

 89%|████████▉ | 1780/2000.0 [00:31<00:03, 58.41it/s]

 89%|████████▉ | 1786/2000.0 [00:31<00:03, 56.58it/s]

 90%|████████▉ | 1792/2000.0 [00:32<00:04, 51.21it/s]

 90%|████████▉ | 1799/2000.0 [00:32<00:03, 54.71it/s]

 90%|█████████ | 1805/2000.0 [00:32<00:03, 51.55it/s]

 91%|█████████ | 1811/2000.0 [00:32<00:03, 53.08it/s]

 91%|█████████ | 1817/2000.0 [00:32<00:03, 52.88it/s]

 91%|█████████ | 1824/2000.0 [00:32<00:03, 54.98it/s]

 92%|█████████▏| 1831/2000.0 [00:32<00:02, 57.28it/s]

 92%|█████████▏| 1838/2000.0 [00:32<00:02, 59.98it/s]

 92%|█████

243